In [2]:
!pip install pyserial

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 90 kB 662 kB/s eta 0:00:01


# 1. Dynamixl Test

In [1]:
from dynamixel_sdk_package import *                    # Uses Dynamixel SDK library

def omniwheel_move(motor_id, speed):
    # Control table address
    ADDR_MX_TORQUE_ENABLE      = 24                 # Control table address is different in Dynamixel model
    ADDR_MX_GOAL_POSITION      = 30
    ADDR_MX_PRESENT_POSITION   = 36
    ADDR_MX_MOVING_SPEED       = 32

    # Protocol version
    PROTOCOL_VERSION            = 1.0               # See which protocol version is used in the Dynamixel
    
    # Default setting
    DXL_ID = motor_id
    #DXL_ID                      = 1                # Dynamixel ID : 1
    BAUDRATE                    = 57600             # Dynamixel default baudrate : 57600
    DEVICENAME                  = '/dev/ttyUSB0'    # Check which port is being used on your controller
                                                    # ex) Windows: "COM1"   Linux: "/dev/ttyUSB0" Mac: "/dev/tty.usbserial-*"

    TORQUE_ENABLE               = 1                 # Value for enabling the torque
    TORQUE_DISABLE              = 0                 # Value for disabling the torque
    DXL_MINIMUM_POSITION_VALUE  = 10                # Dynamixel will rotate between this value
    DXL_MAXIMUM_POSITION_VALUE  = 4000              # and this value (note that the Dynamixel would not move when the position value is out of movable range. Check e-manual about the range of the Dynamixel you use.)
    DXL_MOVING_STATUS_THRESHOLD = 20                # Dynamixel moving status threshold

    index = 0
    dxl_goal_position = [DXL_MINIMUM_POSITION_VALUE, DXL_MAXIMUM_POSITION_VALUE]         # Goal position

    # Initialize PortHandler instance
    # Set the port path
    # Get methods and members of PortHandlerLinux or PortHandlerWindows
    portHandler = PortHandler(DEVICENAME)

    # Initialize PacketHandler instance
    # Set the protocol version
    # Get methods and members of Protocol1PacketHandler or Protocol2PacketHandler
    packetHandler = PacketHandler(PROTOCOL_VERSION)

    # Open port
    if portHandler.openPort():
        #print("Succeeded to open the port")
        pass
    else:
        print("Failed to open the port")
        print("Press any key to terminate...")
        getch()
        quit()

    # Set port baudrate
    if portHandler.setBaudRate(BAUDRATE):
        #print("Succeeded to change the baudrate")
        pass
    else:
        print("Failed to change the baudrate")
        print("Press any key to terminate...")
        getch()
        quit()

    # Enable Dynamixel Torque
    #dxl_comm_result, dxl_error = packetHandler.write1ByteTxRx(portHandler, DXL_ID, ADDR_MX_TORQUE_ENABLE, TORQUE_ENABLE)
    dxl_comm_result, dxl_error = packetHandler.write2ByteTxRx(portHandler, DXL_ID, ADDR_MX_MOVING_SPEED, speed)
    print("dxl_comm_result: " + str(dxl_comm_result))
    if dxl_comm_result != COMM_SUCCESS:
        print("%s" % packetHandler.getTxRxResult(dxl_comm_result))
    elif dxl_error != 0:
        print("%s" % packetHandler.getRxPacketError(dxl_error))
    else:
        pass
        #print("Dynamixel has been successfully connected")

In [2]:
!sudo chmod a+rw /dev/ttyUSB0
!sudo chmod a+rw /dev/ttyTHS0
#omniwheel_move(1,50)

[sudo] password for kimbring2: 


In [2]:
speed = 100
wheel1_speed = 0
wheel2_speed = 0
wheel3_speed = 0
wheel4_speed = 0

# Stop
def stop():
    wheel1_speed = 0
    wheel2_speed = 0
    wheel3_speed = 0
    wheel4_speed = 0
    
    omniwheel_move(0,wheel1_speed)
    omniwheel_move(1,wheel2_speed)
    omniwheel_move(2,wheel3_speed)
    omniwheel_move(3,wheel4_speed)
    #time.sleep(0.2)
    
# Forward
def back():
    wheel1_speed = 1074 + speed
    wheel2_speed = 50 + speed
    wheel3_speed = 50 + speed
    wheel4_speed = 1074 + speed
    
    omniwheel_move(0,wheel1_speed)
    omniwheel_move(1,wheel2_speed)
    omniwheel_move(2,wheel3_speed)
    omniwheel_move(3,wheel4_speed)
    #time.sleep(0.2)
    
# Left
def right():
    wheel1_speed = 1074
    wheel2_speed = 1074
    wheel3_speed = 1074
    wheel4_speed = 1074
    
    omniwheel_move(0,wheel1_speed)
    omniwheel_move(1,wheel2_speed)
    omniwheel_move(2,wheel3_speed)
    omniwheel_move(3,wheel4_speed)
    #time.sleep(0.2)
    
# Right
def left():
    wheel1_speed = 50
    wheel2_speed = 50
    wheel3_speed = 50
    wheel4_speed = 50
    
    omniwheel_move(0,wheel1_speed)
    omniwheel_move(1,wheel2_speed)
    omniwheel_move(2,wheel3_speed)
    omniwheel_move(3,wheel4_speed)
    #time.sleep(0.2)
    
# Back
def forward():
    wheel1_speed = 50 + speed
    wheel2_speed = 1074 + speed
    wheel3_speed = 1074 + speed
    wheel4_speed = 50 + speed
    
    omniwheel_move(0,wheel1_speed)
    omniwheel_move(1,wheel2_speed)
    omniwheel_move(2,wheel3_speed)
    omniwheel_move(3,wheel4_speed)
    #time.sleep(0.2)

In [3]:
forward()

dxl_comm_result: 0
[RxPacketError] Input voltage error!
dxl_comm_result: 0
[RxPacketError] Input voltage error!
dxl_comm_result: 0
[RxPacketError] Input voltage error!
dxl_comm_result: 0
[RxPacketError] Input voltage error!


In [4]:
stop()

dxl_comm_result: 0
[RxPacketError] Input voltage error!
dxl_comm_result: 0
[RxPacketError] Input voltage error!
dxl_comm_result: 0
[RxPacketError] Input voltage error!
dxl_comm_result: 0
[RxPacketError] Input voltage error!


In [5]:
import traitlets
from traitlets.config.configurable import Configurable


class OmniWheelStop(Configurable):
    value = traitlets.Float()

    def __init__(self, *args, **kwargs):
        super(OmniWheelStop, self).__init__(*args, **kwargs)  # initializes traitlets
    
    @traitlets.observe('value')
    def _observe_value(self, change):
        self._write_value(change['new'])

    def _write_value(self, value):
        """Sets motor value between [-1, 1]"""
        stop()

    def _release(self):
        """Stops motor by releasing control"""
        print("_release")

        
class OmniWheelForward(Configurable):
    value = traitlets.Float()

    def __init__(self, *args, **kwargs):
        super(OmniWheelForward, self).__init__(*args, **kwargs)  # initializes traitlets
    
    @traitlets.observe('value')
    def _observe_value(self, change):
        self._write_value(change['new'])

    def _write_value(self, value):
        """Sets motor value between [-1, 1]"""
        forward()

    def _release(self):
        """Stops motor by releasing control"""
        print("_release")

        
class OmniWheelLeft(Configurable):
    value = traitlets.Float()

    def __init__(self, *args, **kwargs):
        super(OmniWheelLeft, self).__init__(*args, **kwargs)  # initializes traitlets
    
    @traitlets.observe('value')
    def _observe_value(self, change):
        self._write_value(change['new'])

    def _write_value(self, value):
        """Sets motor value between [-1, 1]"""
        left()

    def _release(self):
        """Stops motor by releasing control"""
        print("_release")
        
        
class OmniWheelRight(Configurable):
    value = traitlets.Float()

    def __init__(self, *args, **kwargs):
        super(OmniWheelRight, self).__init__(*args, **kwargs)  # initializes traitlets
    
    @traitlets.observe('value')
    def _observe_value(self, change):
        self._write_value(change['new'])

    def _write_value(self, value):
        """Sets motor value between [-1, 1]"""
        right()

    def _release(self):
        """Stops motor by releasing control"""
        print("_release")
        
        
class OmniWheelBack(Configurable):
    value = traitlets.Float()

    def __init__(self, *args, **kwargs):
        super(OmniWheelBack, self).__init__(*args, **kwargs)  # initializes traitlets
    
    @traitlets.observe('value')
    def _observe_value(self, change):
        self._write_value(change['new'])

    def _write_value(self, value):
        """Sets motor value between [-1, 1]"""
        back()

    def _release(self):
        """Stops motor by releasing control"""
        print("_release")

In [6]:
omniwheel_stop = OmniWheelStop()
omniwheel_forward = OmniWheelForward()
omniwheel_left = OmniWheelLeft()
omniwheel_right = OmniWheelRight()
omniwheel_back = OmniWheelBack()

# 2. Solenoid Test

In [7]:
import RPi.GPIO as GPIO
import time

def solenoid_active():
    # Pin Definitions
    output_pin = 18  # BCM pin 18, BOARD pin 12
    
    # Pin Setup:
    GPIO.setmode(GPIO.BCM)  # BCM pin-numbering scheme from Raspberry Pi
    # set pin as an output pin with optional initial state of HIGH
    GPIO.setup(output_pin, GPIO.OUT, initial=GPIO.HIGH)

    print("Starting demo now! Press CTRL+C to exit")
    curr_value = GPIO.HIGH
    time.sleep(1)
    
    print("Outputting {} to pin {}".format(curr_value, output_pin))
    GPIO.output(output_pin, curr_value)
    #curr_value ^= GPIO.HIGH
    GPIO.cleanup()

In [8]:
solenoid_active()

Starting demo now! Press CTRL+C to exit
Outputting 1 to pin 18


In [9]:
class SolenoidKick(Configurable):
    value = traitlets.Float()

    def __init__(self, *args, **kwargs):
        super(SolenoidKick, self).__init__(*args, **kwargs)  # initializes traitlets
    
    @traitlets.observe('value')
    def _observe_value(self, change):
        self._write_value(change['new'])

    def _write_value(self, value):
        """Sets motor value between [-1, 1]"""
        solenoid_active()

    def _release(self):
        """Stops motor by releasing control"""
        print("_release")

In [10]:
solenoid_kick = SolenoidKick()

# 3. Roller Test

In [11]:
import RPi.GPIO as GPIO
import time

def roller_activate(value):
    print("value: " + str(value))
    
    if value != 0:
        # Pin Definitions
        output_pin_1 = 5  # BCM pin 18, BOARD pin 12
        output_pin_2 = 6  # BCM pin 18, BOARD pin 12

        # Pin Setup:
        GPIO.setmode(GPIO.BCM)  # BCM pin-numbering scheme from Raspberry Pi
        # set pin as an output pin with optional initial state of HIGH
        GPIO.setup(output_pin_1, GPIO.OUT, initial=GPIO.LOW)
        GPIO.setup(output_pin_2, GPIO.OUT, initial=GPIO.HIGH)

        #time.sleep(0.5)

        #GPIO.output(5, GPIO.LOW)
        #GPIO.output(6, GPIO.HIGH)

        #time.sleep(0.5)
    else:
        GPIO.cleanup()

In [12]:
roller_activate(5)
time.sleep(0.5)
roller_activate(0)

value: 5
value: 0


In [13]:
class RollerIn(Configurable):
    value = traitlets.Float()

    def __init__(self, *args, **kwargs):
        super(RollerIn, self).__init__(*args, **kwargs)  # initializes traitlets
    
    @traitlets.observe('value')
    def _observe_value(self, change):
        self._write_value(change['new'])

    def _write_value(self, value):
        """Sets motor value between [-1, 1]"""
        roller_activate(value)

    def _release(self):
        """Stops motor by releasing control"""
        print("_release")

In [14]:
roller_in = RollerIn()

# 4. Camera Test

### Create camera instance

Well, right now there's no image presented, because we haven't set the value yet!  We can do this by creating our ``Camera``
class and attaching the ``value`` attribute of the camera to the ``value attribute of the image.

First, let's create the camera instance, we call the ``instance`` method which will create a new camera
if it hasn't been created yet.  If once already exists, this method will return the existing camera.

In [15]:
from jetbot import Camera

camera = Camera.instance()

### Create and display Image widget

First, let's display an ``Image`` widget that we'll use to show our live camera feed.  We'll set the ``height`` and ``width``
to just 300 pixels so it doesn't take up too much space.

> FYI: The height and width only effect the rendering on the browser side, not the native image resolution before network transport from robot to browser.

In [16]:
import ipywidgets as widgets

image = widgets.Image(format='jpeg', width=300, height=300)
display(image)

Image(value=b'', format='jpeg', height='300', width='300')

Our camera class currently only produces values in BGR8 (blue, green, red, 8bit) format, while our image widget accepts values in compressed *JPEG*.
To connect the camera to the image we need to insert the ``bgr8_to_jpeg`` function as a transform in the link.  We do this below

In [17]:
import traitlets
from jetbot import bgr8_to_jpeg

camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)

# 5. Lidar Test

In [18]:
# -*- coding: utf-8 -*
import serial
import time
import threading
from IPython.display import clear_output

ser = serial.Serial("/dev/ttyTHS0", 115200)

def getTFminiData():
    #while True:
    #time.sleep(0.1)
    distance = -1
    
    count = ser.in_waiting
    if count > 8:
        #clear_output(wait=True)
            
        recv = ser.read(9)
        ser.reset_input_buffer()
        # type(recv), 'str' in python2(recv[0] = 'Y'), 'bytes' in python3(recv[0] = 89)
        # type(recv[0]), 'str' in python2, 'int' in python3 

        if recv[0] == 0x59 and recv[1] == 0x59:     #python3
            distance = recv[2] + recv[3] * 256
            strength = recv[4] + recv[5] * 256
            #print('(', distance, ',', strength, ')')
            ser.reset_input_buffer()

        if recv[0] == 'Y' and recv[1] == 'Y':     #python2
            lowD = int(recv[2].encode('hex'), 16)
            highD = int(recv[3].encode('hex'), 16)
            lowS = int(recv[4].encode('hex'), 16)
            highS = int(recv[5].encode('hex'), 16)
            distance = lowD + highD * 256
            strength = lowS + highS * 256
        
        #print("distance: " + str(distance))

        return distance
    else:
        return 0
    
    
def _get_distance():
    while True:
        lidar_value = getTFminiData()
        time.sleep(0.5)
        
        if lidar_value != -1:
            print("lidar_value: " + str(lidar_value))
        else:
            break

In [20]:
getTFminiData()

189

In [21]:
from ipywidgets import IntProgress
from IPython.display import display
import time

max_count = 100

f = IntProgress(min=0, max=max_count) # instantiate the bar
display(f) # display the bar

count = 0
while True:
    #f.value += 1 # signal to increment the progress bar
    #lidar_value = getTFminiData()
    #print("lidar_value: " + str(lidar_value))
    f.value = getTFminiData()
    time.sleep(0.1)
    count += 1

IntProgress(value=0)

KeyboardInterrupt: 

# 6. Infrared Test

In [22]:
import RPi.GPIO as GPIO
import time


def get_infrared_data():
    # Pin Definitions
    input_pin = 27  # BCM pin 18, BOARD pin 12
    
    # Pin Setup:
    GPIO.setmode(GPIO.BCM)  # BCM pin-numbering scheme from Raspberry Pi
    GPIO.setup(input_pin, GPIO.IN)  # set pin as an input pin
    #print("Starting demo now! Press CTRL+C to exit")
    value = GPIO.input(input_pin)
    #print("value: " + str(value))

    GPIO.cleanup()
    
    return value

In [23]:
get_infrared_data()

1

In [24]:
from ipywidgets import Button

b1 = Button(description='Infrared')
display(b1)
#b1.style.button_color = 'lightgreen'

while True:
    infrared_value = get_infrared_data()
    #print("infrared_value: " + str(infrared_value))
    if infrared_value == 0:
        b1.style.button_color = 'red'
    else:
        b1.style.button_color = 'lightgreen'
    
    time.sleep(0.1)

#b1.style.button_color = 'red'
#b1

Button(description='Infrared', style=ButtonStyle())

KeyboardInterrupt: 

# Teleoperation

In this example we'll control the Jetbot remotely with a gamepad controller connected to our web browser machine.

In [7]:
!jupyter nbextension enable --py widgetsnbextension

Collisions detected in /home/kimbring2/.jupyter/jupyter_notebook_config.py and /home/kimbring2/.jupyter/jupyter_notebook_config.json config files. /home/kimbring2/.jupyter/jupyter_notebook_config.json has higher priority: {
      "NotebookApp": {
        "password": "'sha1:60f3ac9aec93:be2d6048e9b1e7ae0f1ccbad9d746734bf5c3797' ignored, using 'argon2:$argon2id$v=19$m=10240,t=10,p=8$jlnJIY+q0X2G1zF/0ul1PA$ksLJmHW7IX7ezNJthhwwZg'"
      }
    }
Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK


In [25]:
import ipywidgets.widgets as widgets

controller = widgets.Controller(index=0)  # replace with index of your controller

display(controller)

Controller()

Even if the index is correct, you may see the text ``Connect gamepad and press any button``.  That's because the gamepad hasn't
registered with this notebook yet.  Press a button and you should see the gamepad widget appear above.

### Create gamepad controller

The first thing we want to do is create an instance of the ``Controller`` widget, which we'll use to drive our robot.
The ``Controller`` widget takes a ``index`` parameter, which specifies the number of the controller.  This is useful in case you
have multiple controllers attached, or some gamepads *appear* as multiple controllers.  To determine the index
of the controller you're using,

1. Visit [http://html5gamepad.com](http://html5gamepad.com).  
2. Press buttons on the gamepad you're using
3. Remember the ``index`` of the gamepad that is responding to the button presses

Next, we'll create and display our controller using that index.

### Connect gamepad controller to robot motors

Now, even though we've connected our gamepad, we haven't yet attached the controls to our robot!  The first, and most simple control
we want to attach is the motor control.  We'll connect that to the left and right vertical axes using the ``dlink`` function.  The
``dlink`` function, unlike the ``link`` function, allows us to attach a transform between the ``source`` and ``target``.  Because
the controller axes are flipped from what we think is intuitive for the motor control, we'll use a small *lambda* function to
negate the value.

> WARNING: This next cell will move the robot if you touch the gamepad controller axes!

In [28]:
from jetbot import Robot
import traitlets

#robot = Robot()

solenoid_link = traitlets.dlink((controller.buttons[4], 'value'), (solenoid_kick, 'value'), transform=lambda x: -x)
roller_link = traitlets.dlink((controller.axes[1], 'value'), (roller_in, 'value'), transform=lambda x: -x)

stop_link = traitlets.dlink((controller.buttons[5], 'value'), (omniwheel_stop, 'value'), transform=lambda x: -x)
forward_link = traitlets.dlink((controller.buttons[0], 'value'), (omniwheel_forward, 'value'), transform=lambda x: -x)
left_link = traitlets.dlink((controller.buttons[1], 'value'), (omniwheel_left, 'value'), transform=lambda x: -x)
right_link = traitlets.dlink((controller.buttons[2], 'value'), (omniwheel_right, 'value'), transform=lambda x: -x)
back_link = traitlets.dlink((controller.buttons[3], 'value'), (omniwheel_back, 'value'), transform=lambda x: -x)

Awesome! Our robot should now respond to our gamepad controller movements.  Now we want to view the live video feed from the camera!

### Connect Camera to Image widget

You should now see the live video feed shown above!

> REMINDER:  You can right click the output of a cell and select ``Create New View for Output`` to display the cell in a separate window.

### Stop robot if network disconnects

You can drive your robot around by looking through the video feed. But what if your robot disconnects from Wifi?  Well, the motors would keep moving and it would keep trying to stream video and motor commands.  Let's make it so that we stop the robot and unlink the camera and motors when a disconnect occurs.

In [6]:
from jetbot import Heartbeat


def handle_heartbeat_status(change):
    if change['new'] == Heartbeat.Status.dead:
        camera_link.unlink()
        left_link.unlink()
        right_link.unlink()
        robot.stop()

heartbeat = Heartbeat(period=0.5)

# attach the callback function to heartbeat status
heartbeat.observe(handle_heartbeat_status, names='status')

If the robot disconnects from the internet you'll notice that it stops.  You can then re-connect the camera and motors by re-creating the links with the cell below

In [7]:
# only call this if your robot links were unlinked, otherwise we'll have redundant links which will double
# the commands transfered

left_link = traitlets.dlink((controller.axes[0], 'value'), (robot.left_motor, 'value'), transform=lambda x: -x)
right_link = traitlets.dlink((controller.axes[1], 'value'), (robot.right_motor, 'value'), transform=lambda x: -x)
camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)

### Save snapshots with gamepad button

Now, we'd like to be able to save some images from our robot.  Let's make it so the right bumper (index 5) saves a snapshot of the current live image.  We'll save the images in the ``snapshots/`` directory, with a name that is guaranteed to be unique using the ``uuid`` python package.  We use the ``uuid1`` identifier, because this also encodes the date and MAC address which we might want to use later.

In [15]:
import uuid
import subprocess

subprocess.call(['mkdir', '-p', 'snapshots'])

snapshot_image = widgets.Image(format='jpeg', width=300, height=300)

def save_snapshot(change):
    # save snapshot when button is pressed down
    if change['new']:
        file_path = 'snapshots/' + str(uuid.uuid1()) + '.jpg'
        
        # write snapshot to file (we use image value instead of camera because it's already in JPEG format)
        with open(file_path, 'wb') as f:
            f.write(image.value)
            
        # display snapshot that was saved
        snapshot_image.value = image.value


controller.buttons[3].observe(save_snapshot, names='value')

display(widgets.HBox([image, snapshot_image]))
display(controller)

Controller(axes=(Axis(value=0.0), Axis(value=0.0), Axis(value=0.0), Axis(value=0.0), Axis(value=0.0), Axis(val…

### Conclusion

That's it for this example, have fun!